In [16]:
import os 
import cv2 
import numpy as np 
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np 
from tensorflow.keras import layers
from scipy.io import loadmat
from tensorflow import keras
import tensorflow as tf 
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageFile
from PIL import ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [25]:
input_dir = 'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_image2'
target_dir = 'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_mask4'
img_size = (224,224)
num_classes = 7
batch_size = 16

input_img_paths = sorted([os.path.join(input_dir,fname)
                         for fname in os.listdir(input_dir)
                         if fname.endswith('.jpg')])

target_img_paths = sorted([os.path.join(target_dir,fname)
                         for fname in os.listdir(target_dir)
                         if fname.endswith('.png') and not fname.startswith('.')])

In [26]:
# RGB value to class label mapping
RGB_TO_LABEL = {
    '[0, 0, 0]': 0,  # Background class
    '[255, 0, 0]': 1,
    '[0, 0, 255]': 2,
    '[0, 255, 0]': 3,
    '[255, 255, 0]': 4,
    '[128, 0, 128]': 5,
    '[255, 165, 0]': 6,
}

In [27]:
def load_mask_img(path, target_size, rgb_to_label):
    # Load image in RGB mode
    img = load_img(path, target_size=target_size)
    img_array = img_to_array(img).astype(int)  # Convert RGB values to integers
    # Initialize an empty array to hold the labels
    label_array = np.zeros(img_array.shape[:-1], dtype=np.uint8)
    # Iterate over each pixel and replace RGB value with corresponding label
    for i in range(label_array.shape[0]):
        for j in range(label_array.shape[1]):
            rgb = img_array[i, j, :3]
            rgb_str = str(rgb.tolist())
            if rgb_str in rgb_to_label:
                label_array[i, j] = rgb_to_label[rgb_str]
            else:
                raise ValueError(f"RGB value {rgb_str} not in RGB_TO_LABEL dictionary.")
    return label_array

In [28]:
class SkinDiseaseDataset(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i:i+self.batch_size]
        batch_target_img_paths = self.target_img_paths[i:i+self.batch_size]
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype='float32')
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            x[j] = img_to_array(img) / 255.  # input normalization
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype='uint8')
        for j, path in enumerate(batch_target_img_paths):
            label_array = load_mask_img(path, self.img_size, RGB_TO_LABEL)
            y[j] = np.expand_dims(label_array, -1)

        return x, y

In [29]:
import random 
val_samples = 600
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)

train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

train_gen = SkinDiseaseDataset(batch_size, img_size, train_input_img_paths, train_target_img_paths)
val_gen = SkinDiseaseDataset(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [30]:
x, y = train_gen[0]  # 첫 번째 배치 가져오기
print(x.shape, y.shape)
print(np.min(x), np.max(x), np.min(y), np.max(y)) 

(16, 224, 224, 3) (16, 224, 224, 1)
0.0 1.0 0 5


In [13]:
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

{0: 802816}


In [11]:
class_weights = 1.0 / np.array(counts)
class_weights = class_weights / np.sum(class_weights)

In [12]:
def weighted_cross_entropy(class_weights): 
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.nn.softmax(y_pred)  # softmax 활성화 함수 적용
        loss_per_class = -tf.reduce_sum(y_true * tf.math.log(y_pred + 1e-5), axis=[1, 2])  # 상수를 1e-5로 변경
        weighted_loss = loss_per_class * class_weights
        return tf.reduce_mean(weighted_loss)
    return loss

In [13]:
loss_function = weighted_cross_entropy(class_weights)

In [14]:
def convolution_block(block_input, num_filters=256, kernel_size=3, 
                     dilation_rate=1, padding='same', use_bias=False):
    x = layers.Conv2D(num_filters, kernel_size=kernel_size, 
                      dilation_rate=dilation_rate, padding='same', 
                      use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)

In [15]:
def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(size=(dims[-3]// x.shape[1], dims[-2]// x.shape[2]),
                                         interpolation='bilinear')(x)
    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
    
    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    
    output = convolution_block(x, kernel_size=1)
    return output

In [16]:
def DeeplabV3(img_size, num_classes):
    model_input = keras.Input(shape=img_size + (3,))
    resnet50 = keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=model_input)
    
    x = resnet50.get_layer('conv4_block6_2_relu').output
    x = DilatedSpatialPyramidPooling(x)
    
    input_a = layers.UpSampling2D(size=(img_size[0] // 4 // x.shape[1],
                                       img_size[1] // 4 // x.shape[2]),
                                       interpolation='bilinear')(x)
    input_b = resnet50.get_layer('conv2_block3_2_relu').output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)
    
    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(size=(img_size[0] // x.shape[1],
                                 img_size[1] // x.shape[2]),
                           interpolation='bilinear')(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1,1), padding='same')(x)
    
    return keras.Model(inputs=model_input, outputs=model_output)

In [17]:
model = DeeplabV3(img_size=img_size, num_classes=num_classes)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-6), loss=loss_function, metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
337/337 [==============================] - ETA: 0s - loss: 844.0404 - accuracy: 0.1562